In [1]:
import sys
#!{sys.executable} -m pip install pandas

In [2]:
import pandas as pd
import numpy as np
import abydos
from abydos import distance

# General Instruction

- Cleanup dataset based on the information that is given:
You need to clean the dataset according to the information that is given to you. This means that there are problems with the dataset that need to be fixed, and you should use the information given to you to determine what those problems are and how to fix them.

- Each case has different data quality problems, there will be hint and additional information that can help you understand the problem:
Each row in the dataset may have different data quality problems. There will be hints and additional information provided to help you understand what the specific problem is with each row.

- You can do any approach on cleaning the data, but you should clean the instructed column only:
You have the freedom to use any approach to clean the data, but you should only clean the instructed column. This means that you should not modify any other columns in the dataset, or add or remove any rows.

- Do not create new column or remove any column. Also do not create new row, or remove any row:
- You are not allowed to create new columns or remove any columns from the dataset. You are also not allowed to add or remove any rows.

- Each column will have a flag column something equivalent to <column\_name>\_flag. This column can be used to flag the row if you want to not include it to the downstream task. 0: safe_flag, 1: delete_flag, 2: null_flag (if you want to still include the row with null treatment). You can also add a new category but please add justification and explanation of the new category, there are three categories you can use:
safe_flag (0): this row is safe to use in downstream tasks
delete_flag (1): this row should be deleted and not used in downstream tasks
null_flag (2): this row can be included in downstream tasks but with null treatment.
You can also add a new category, but you need to provide a justification and an explanation for the new category. It is worth to note that the completeness of the dataset is also matter, so try not to flag to many things, and do your best to clean the values.

- For each data cleaning task, we have provided a function that represents the goal of the cleaning. For example, clean_duplicate_id(df) is the function for removing duplicate ID values. These functions take a DataFrame as input and return the cleaned version of the DataFrame.

    In each chunk of data cleaning task, you will see the following three parts:

    1. The clean_<name> function that performs the specific cleaning task.
    2. The execution of the cleaning function on the DataFrame.
    3. A checking part to help you evaluate the effectiveness of the cleaning.
    
  While you can create new cells and add additional code, the cleaning must be performed through the provided cleaning functions. You can adjust the order of the cleaning steps, but please try to move the whole chunks of code to avoid any errors.

The cleaning task will be considered complete if this notebook can be run sequentially by executing "restart and runall"




# Purpose
The purpose of this dataset is to conduct exploratory analysis of the listings and create a prediction model for listing price using some columns from the dataset. This means that the dataset is intended to be used to explore the characteristics and features of the listings, and to build a model that can predict the price of a listing based on certain variables in the dataset. The goal is to gain insights into the factors that influence the price of a listing and to develop a model that can accurately predict listing prices based on those factors.

# Columns and Dataset Description
- id: a unique identifier for each listing.
- name: the name or title of the listing, as provided by the host.
- host_id: a unique identifier for each host.
- host_name: the name of the host who listed the property.
- neighbourhood_group: the larger geographic area in which the listing is located (e.g. a borough or group of neighborhoods).
- neighbourhood: the specific neighborhood in which the listing is located.
- latitude: the latitude coordinate of the listing.
- longitude: the longitude coordinate of the listing.
- room_type: the type of space that is being listed (e.g. an entire apartment, a private room, a shared room).
- price: the nightly price of the listing, in the currency specified in the dataset.
- minimum_nights: the minimum number of nights that a guest must book the listing for.
- number_of_reviews: the total number of reviews that the listing has received.
- last_review: the date of the most recent review of the listing.
- reviews_per_month: the average number of reviews per month that the listing has received.
- calculated_host_listings_count: the total number of listings that the host has on Airbnb.
- availability_365: the number of days per year that the listing is available for booking.
- number_of_reviews_ltm: the total number of reviews that the listing has received in the last 12 months.
- license: a license number for the listing, if applicable (this column may not be present in all versions of the dataset).

Besides the columns above, there are columns pre-defined for flagging the rows based on particular data cleaning context:
- id_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the id column (duplicate).
- host_id_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the host_id column.
- neighbourhood_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the neighbourhood column.
- latitude_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the latitude column.
- longitude_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the longitude column.
- minimum_nights_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the minimum_nights column.
- number_of_reviews_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the number_of_reviews column.
- last_review_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the last_review column.
- room_type_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the room_type column.

# Load Data

In [3]:
airbnb_pd = pd.read_csv("chicago_vert_2_a.csv")

In [4]:
airbnb_pd.neighbourhood.unique()

array(['Chicago, Illinois, United States',
       'Chicago, Illinois, UnitejduV States', nan,
       'Chicago, vIlljinois, Uniated States',
       'Chicagoy,MK Illinois, United States',
       'Chicago, IllinoisTc, United SRtates',
       'ChiNciacgo, Illinois, United States',
       'rChicago, Illinosis, United SEtates',
       'Chicago, GIllinoisK, United Strates',
       'Chicago, Illinois,N fOUnited States',
       'hChicago, Illinoiss, Unilted States',
       'aChicago, Illinois, qUnited Staktes',
       'ChVicago, Illinois, UnitqedV States',
       'Chicago, Illinois, Unipted eStatPes',
       'Chicago, Illinois, Ubnited MStaetes', 'lbndan',
       'rChicago, Illinois, UnitedW Satates',
       'NCQhicago, Illinois, UniteMd States',
       'ChiPcago, Illinois,y UnTited States',
       'Chicago, Illinois, United  Strlates', 'ncaNwn',
       'CShicagGo, Illinois, Unbited States',
       'Chicagos, Illinoxis,U United States',
       'Chicago, ImlulYinois, United States',
       'Chbi

In [5]:
airbnb_pd.head()

,Unnamed: 0,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,...,reviews_per_month,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
0,7738,7126,https://www.airbnb.com/rooms/7126,20230319041143,2023-03-19,city scrape,Tiny Studio Apartment 94 Walk Score,A very small studio in a wonderful neighborhoo...,"Ukrainian Village was just named ""Hottest Neig...",https://a0.muscache.com/pictures/51073/16c81c7...,...,2.89,0,0,0,0,0,0,0,0,0
1,7746,84042,https://www.airbnb.com/rooms/84042,20230319041143,2023-03-19,city scrape,The Explorer Room,This is a peaceful/private bedroom (in a share...,Accessible to everything Chicago has to offer....,https://a0.muscache.com/pictures/13143585/e400...,...,0.85,0,0,0,0,0,0,0,0,0
2,7651,145659,https://www.airbnb.com/rooms/145659,20230319041143,2023-03-19,city scrape,Trendy Roscoe Village 3BR/2BR walk to shops,Awesome 3 bedroom/2 bathroom in one of Chicago...,Trendy Roscoe Village offers a variety of bout...,https://a0.muscache.com/pictures/65ff405d-3f96...,...,0.65,0,0,0,0,0,0,0,0,0
3,7659,264547,https://www.airbnb.com/rooms/264547,20230319041143,2023-03-19,city scrape,Steps From the Best Chicago Offers,Hotel type amenities (in excess) upon arrival....,"It is Chicago's hottest area, centrally locate...",https://a0.muscache.com/pictures/2617984/d552d...,...,0.51,0,0,0,0,0,0,0,0,0
4,7662,290155,https://www.airbnb.com/rooms/290155,20230319041143,2023-03-19,previous scrape,15 Minutes to Everything. Skyline View. Quiet.,Discounts and free nights for 5 nights or more...,"Our area is quiet, and safe. We have great l...",https://a0.muscache.com/pictures/27e377b7-b7a1...,...,0.37,0,0,0,0,0,0,0,0,0


In [6]:
airbnb_pd.describe()

,Unnamed: 0,id,scrape_id,host_id,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,...,reviews_per_month,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
count,1542.000000,1.542000e+03,1.542000e+03,1.542000e+03,1542.000000,1542.000000,0.0,1.535000e+03,1.532000e+03,1542.000000,...,1273.000000,1542.0,1542.0,1542.0,1542.0,1542.0,1542.0,1542.0,1542.0,1542.0
mean,3500.671206,2.083386e+17,2.023032e+13,1.499255e+08,537.351492,615.239300,NaN,1.644628e+06,-2.860923e+13,4.243191,...,1.677604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,2033.600736,3.123278e+17,7.111681e-01,1.411985e+08,1489.252081,1690.948556,NaN,2.394459e+07,5.001295e+14,3.015405,...,1.614414,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,7.000000,7.126000e+03,2.023032e+13,2.153000e+03,0.000000,0.000000,NaN,4.165895e+01,-8.768228e+15,1.000000,...,0.010000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,1777.750000,2.926225e+07,2.023032e+13,3.328929e+07,1.000000,2.000000,NaN,4.187583e+01,-8.769192e+01,2.000000,...,0.460000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,3462.000000,4.702248e+07,2.023032e+13,1.074344e+08,5.000000,7.000000,NaN,4.189627e+01,-8.766223e+01,4.000000,...,1.340000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,5180.000000,5.980440e+17,2.023032e+13,2.314393e+08,23.000000,33.000000,NaN,4.191340e+01,-8.763341e+01,6.000000,...,2.510000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,7746.000000,7.854264e+17,2.023032e+13,4.881240e+08,4807.000000,8342.000000,NaN,4.190293e+08,-8.753409e+01,16.000000,...,16.570000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# cleanup duplicate id
The ID column must contain unique values. If there are any duplicate values in this column, you will need to take action to ensure that each ID is unique. You can do this by either fixing the duplicates (if you want to keep them) or by flagging them for removal (1) using the id_flag column.

In [7]:
def clean_duplicate_id(df):
    #raise Exception("not yet have implementation")
    # do something here
    df["id_flag"] = df.duplicated(subset=["id"], keep="first").astype(int)
    return df

In [8]:
airbnb_pd = clean_duplicate_id(airbnb_pd)

In [9]:
# Check id_flag
airbnb_pd["id_flag"].value_counts()

0    1469
1      73
Name: id_flag, dtype: int64

# Duplicate IDS checking 
To ensure that all ID values in the dataset are unique, you should check for duplicate IDs. When you run the query to check for duplicates, there should be no rows returned, indicating that there are no duplicate ID values present in the dataset.

In [10]:
dup_ids = airbnb_pd[airbnb_pd.id_flag==0]
dup_ids = dup_ids.groupby("id").count()[["name"]].reset_index()
dup_ids = dup_ids[dup_ids.name>1]
dup_ids

,id,name


# cleanup inconsistent host id
Each host_id value in the dataset should be associated with only one host_name. However, there may be inconsistencies in the dataset where a host_id is associated with different host_name values.

To clean this up, you can either change the host_name value to a consistent value based on information in the dataset, or flag the host_id_flag column to indicate that the row should be removed from downstream tasks.

For example, if you find that a host_id is associated with multiple host_name values, you may want to investigate further to determine which host_name is correct. If one of the host_name values is clearly incorrect (e.g., a misspelling or a name that does not match the owner of the property), you could update the host_name value to the correct value.

Alternatively, if you cannot determine the correct host_name value, or if you want to exclude the row from downstream tasks for other reasons, you can flag the host_id_flag column with a value of 1 to indicate that the row should be removed.

In [11]:
def clean_host_id(df):
    #raise Exception("not yet have implementation")
    # do something here
    
    
    return df

In [12]:
airbnb_pd = clean_host_id(airbnb_pd)

# Inconsistent Host ID checking 

This query should return zero rows once you implement the cleaning process

In [13]:
airbnb_pd[airbnb_pd.host_id==483146]

,Unnamed: 0,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,...,reviews_per_month,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag


In [14]:
dup_host_id = airbnb_pd[airbnb_pd.host_id_flag==0]
dup_host_id = dup_host_id.groupby(["host_id","host_name"]).count()[["id"]].reset_index()
dup_host_id = dup_host_id.groupby("host_id").count()["id"].reset_index()
dup_host_id[dup_host_id["id"]>1]

,host_id,id
7,459291,2
10,683529,2
21,1385332,2
39,2619611,2
49,3571119,2
...,...,...
794,442225178,2
802,448983594,2
808,457609185,2
815,467078606,2


# cleanup neighbourhood
The neighbourhood column in the dataset should contain values that match the neighbourhoods defined in the official neighbourhood_list. However, there may be some values in the neighbourhood column that are incorrect due to errors or noise in the data.

To clean up the neighbourhood column, you can try to match each value in the column to a valid neighbourhood in the neighbourhood_list using a string distance function such as abydos. If you can successfully match a value in the neighbourhood column to a neighbourhood in the neighbourhood_list, you can replace the value in the dataset with the correct neighbourhood name.

However, if you are unsure about how to clean up a particular value in the neighbourhood column, or if you cannot match the value to a valid neighbourhood in the neighbourhood_list, you can flag the row for deletion by setting the neighbourhood_flag column to a value of 1. If the value in the neighbourhood column is null and you cannot make a determination based on other information in the dataset, you can set the neighbourhood_flag column to a value of 2 to indicate that the row should be included but the neighbourhood value is null.

You can also use the latitude and longitude columns in the dataset to help match values in the neighbourhood column to valid neighbourhoods in the neighbourhood_list. However, you should be aware that the latitude and longitude values may also contain errors or noise, so you should exercise caution when using these columns to clean up the neighbourhood column.

In [15]:
neighbourhood_list = [ 'Hyde Park', 'West Town', 'Lincoln Park', 'Near West Side', 'Lake View',    'Dunning', 'Rogers Park', 'Logan Square', 'Uptown', 'Edgewater',    'North Center', 'Albany Park', 'West Ridge', 'Pullman', 'Irving Park',    'Beverly', 'Lower West Side', 'Near South Side', 'Near North Side',    'Grand Boulevard', 'Bridgeport', 'Humboldt Park', 'Chatham', 'Kenwood',    'Loop', 'West Lawn', 'Lincoln Square', 'Woodlawn', 'Avondale',    'Forest Glen', 'Portage Park', 'East Garfield Park', 'Washington Park',    'North Lawndale', 'Armour Square', 'South Lawndale', 'South Shore',    'Morgan Park', 'South Deering', 'West Garfield Park', 'Hermosa',    'Mckinley Park', 'Douglas', 'Hegewisch', 'West Elsdon', 'Norwood Park',    'Garfield Ridge', 'Austin', 'Belmont Cragin', 'Jefferson Park', 'Ashburn',    'Greater Grand Crossing', 'North Park', 'Oakland', 'Archer Heights',    'Edison Park', 'Englewood', 'Ohare', 'Brighton Park', 'Chicago Lawn',    'New City', 'South Chicago', 'Mount Greenwood', 'Montclare', 'Roseland',    'West Englewood', 'Calumet Heights', 'Auburn Gresham', 'Fuller Park',    'Avalon Park', 'Burnside', 'Clearing', 'Gage Park', 'West Pullman',    'Washington Heights', 'East Side']
print(neighbourhood_list)

['Hyde Park', 'West Town', 'Lincoln Park', 'Near West Side', 'Lake View', 'Dunning', 'Rogers Park', 'Logan Square', 'Uptown', 'Edgewater', 'North Center', 'Albany Park', 'West Ridge', 'Pullman', 'Irving Park', 'Beverly', 'Lower West Side', 'Near South Side', 'Near North Side', 'Grand Boulevard', 'Bridgeport', 'Humboldt Park', 'Chatham', 'Kenwood', 'Loop', 'West Lawn', 'Lincoln Square', 'Woodlawn', 'Avondale', 'Forest Glen', 'Portage Park', 'East Garfield Park', 'Washington Park', 'North Lawndale', 'Armour Square', 'South Lawndale', 'South Shore', 'Morgan Park', 'South Deering', 'West Garfield Park', 'Hermosa', 'Mckinley Park', 'Douglas', 'Hegewisch', 'West Elsdon', 'Norwood Park', 'Garfield Ridge', 'Austin', 'Belmont Cragin', 'Jefferson Park', 'Ashburn', 'Greater Grand Crossing', 'North Park', 'Oakland', 'Archer Heights', 'Edison Park', 'Englewood', 'Ohare', 'Brighton Park', 'Chicago Lawn', 'New City', 'South Chicago', 'Mount Greenwood', 'Montclare', 'Roseland', 'West Englewood', 'Calu

In [16]:
def clean_neighbourhood(df):
    #raise Exception("not yet have implementation")
    # do something here
    print("not yet have implementation")
        
    return df

In [17]:
airbnb_pd = clean_neighbourhood(airbnb_pd)

not yet have implementation


# Neighbourhood checking

This query should return zero rows once you implement the cleaning process

In [18]:
neighbourhood_check = airbnb_pd[airbnb_pd.neighbourhood_flag==0]
neighbourhood_check = neighbourhood_check[neighbourhood_check.neighbourhood.apply(lambda x:x not in neighbourhood_list)]
neighbourhood_check[["id","neighbourhood"]]

,id,neighbourhood
0,7126,"Chicago, Illinois, United States"
1,84042,"Chicago, Illinois, United States"
2,145659,"Chicago, Illinois, United States"
3,264547,"Chicago, Illinois, United States"
4,290155,"Chicago, Illinois, United States"
...,...,...
1537,782137041139805370,NaN
1538,782628636832878491,"Chicago, Illinois, United States"
1539,782643895516370805,NaN
1540,785423932330914663,"Chicago, Illinois, United States"


# cleanup latitude and longitude
The latitude and longitude values in the dataset must fall within the range of -90 to +90 for latitude and -180 to +180 for longitude to ensure that they meet the criteria for analysis. We have provided a check number function to validate the latitude and longitude columns. Any values outside of these ranges should be cleaned to meet the criteria.

If you are unsure what to do with a value or if it is a null value, you can flag the row for deletion by setting latitude_flag or longitude_flag to 1 or 2, respectively.

In [19]:
def check_number(x,start=-90,end=90):
    try:
        temp_x = float(x)
        return start <= temp_x <= end
    except:
        return False

In [20]:
# lat_check_pd = airbnb_pd[airbnb_pd.latitude_flag==0]
# lat_check_pd = lat_check_pd[lat_check_pd.latitude.apply(lambda x:check_number(x,-90,90))==False]
# for nbh in lat_check_pd["neighbourhood"].unique():
#     min_pd = airbnb_pd[["id","latitude", "longitude", "neighbourhood"]].copy()
#     min_pd = min_pd[(min_pd["latitude"] >= -90) & (min_pd["latitude"] <= 90)]
#     print("Wrong: ", lat_check_pd[lat_check_pd["neighbourhood"] == nbh][["id","latitude", "longitude", "neighbourhood"]])
#     print("Correct (avg): ", min_pd[min_pd["neighbourhood"] == nbh]["latitude"].mean())
#     print("***"*30)

In [21]:
def adjust_decimal(x,dec_pos=2):
    #raise Exception("not yet have implementation")
    # do something here
    # must be a floating point
    x = float(x)
    abs_x = abs(x)
    num_str = str(abs_x)
    decimal_pos = num_str.find('.')
    if decimal_pos == -1:
        #consider everything
        digits_before_decimal = len(num_str)
    else:
        digits_before_decimal = decimal_pos
    divisor = 10**(digits_before_decimal - dec_pos)
    
    return x / divisor
    
    #how many values behind coma
    
    return df

In [22]:
def clean_latitude(df):
    #raise Exception("not yet have implementation")
    # do something here
    df["latitude"] = df["latitude"].apply(lambda x: x if check_number(x) else adjust_decimal(x))
    df["latitude_flag"] = df["latitude"].apply(lambda x: 2 if pd.isnull(x) else x)
    df["latitude_flag"] = df["latitude"].apply(lambda x: 0 if check_number(x) else 1)
    return df

In [23]:
airbnb_pd = clean_latitude(airbnb_pd)

In [24]:
airbnb_pd.latitude

0       41.901660
1       41.896150
2       41.943420
3       41.884130
4       41.827120
          ...    
1537    41.870469
1538    41.896820
1539    41.901460
1540    41.890516
1541    41.894529
Name: latitude, Length: 1542, dtype: float64

# Latitude checking

This query should return zero rows once you implement the cleaning process

In [25]:
lat_check_pd = airbnb_pd[airbnb_pd.latitude_flag==0]
lat_check_pd = lat_check_pd[lat_check_pd.latitude.apply(lambda x:check_number(x,-90,90))==False]
lat_check_pd[["id","latitude", "longitude", "neighbourhood"]]

,id,latitude,longitude,neighbourhood


In [26]:
def clean_longitude(df):
    #raise Exception("not yet have implementation")
    # do something here
    
    df["longitude"] = df["longitude"].apply(lambda x: x if check_number(x) else adjust_decimal(x))
    df["longitude_flag"] = df["longitude"].apply(lambda x: 0 if check_number(x,-180,180) else 2)
        
    return df

In [27]:
airbnb_pd = clean_longitude(airbnb_pd)

In [28]:
airbnb_pd.groupby("longitude_flag").count()

,Unnamed: 0,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,...,calculated_host_listings_count_shared_rooms,reviews_per_month,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
longitude_flag,,,,,,,,,,,,,,,,,,,,,
0,1532,1532,1532,1532,1532,1532,1531,1532,1103,1532,...,1532,1264,1532,1532,1532,1532,1532,1532,1532,1532
2,10,10,10,10,10,10,9,10,8,10,...,10,9,10,10,10,10,10,10,10,10


# Longitude checking

This query should return zero rows once you implement the cleaning process

In [29]:
lon_check_pd = airbnb_pd[airbnb_pd.longitude_flag==0]
lon_check_pd = lon_check_pd[lon_check_pd.longitude.apply(lambda x:check_number(x,-180,180))==False]
lon_check_pd[["id","longitude"]]

,id,longitude


# cleanup room type
The "room_type" column in the dataset should contain one of the values defined in the list of allowed_room_type provided by the authority: ['Entire home/apt', 'Private room', 'Shared room', 'Hotel room']. Any value outside of this list needs to be adjusted to one of the allowed values.

If you are unsure about how to adjust the value or cannot find a suitable value, you can flag the row for deletion by setting the value of room_type_flag to 1. If the "room_type" column has a null value and you cannot decide on an appropriate value, you can set the value of room_type_flag to 2.

In [30]:
allowed_room_type = ['Entire home/apt', 'Private room', 'Shared room', 'Hotel room']

In [31]:
def clean_room_type(df):
    #raise Exception("not yet have implementation")
    # do something here
    # nikolaus add room type cleaning based on curator 1 algorithm
    airbnb_pd['room_type'] = airbnb_pd['room_type'].replace({'Apartment': 'Entire home/apt'})
    df.loc[~df['room_type'].isin(allowed_room_type), 'room_type_flag'] = 1
    df.loc[df['room_type'].isna(), 'room_type_flag'] = 2 
    return df

In [32]:
airbnb_pd = clean_room_type(airbnb_pd)

In [33]:
airbnb_pd.groupby("room_type_flag").count()

,Unnamed: 0,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,...,calculated_host_listings_count_shared_rooms,reviews_per_month,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag
room_type_flag,,,,,,,,,,,,,,,,,,,,,
0,1533,1533,1533,1533,1533,1533,1532,1533,1103,1533,...,1533,1265,1533,1533,1533,1533,1533,1533,1533,1533
2,9,9,9,9,9,9,8,9,8,9,...,9,8,9,9,9,9,9,9,9,9


# room_type checking

This query should return zero rows once you implement the cleaning process

In [34]:
room_type_pd = airbnb_pd[airbnb_pd.room_type_flag==0]
room_type_pd = room_type_pd[room_type_pd.room_type.apply(lambda x: x not in allowed_room_type)]
room_type_pd[["id","room_type"]]

,id,room_type


# cleanup minimum_nights and number_of_reviews

The columns "minimum_nights" and "number_of_reviews" should both be integer values. "minimum_nights" should be a value between 1 and the number of days in a year (365), while "number_of_reviews" should be a value between 0 and 999999.

To check if these columns meet the criteria, we have provided a "check_integer" function. Any values that do not meet the criteria should be cleaned to meet the criteria for analysis.

If you are unsure what to do with a value or if it is a null value, you can flag the row for deletion by setting "minimum_nights_flag" or "number_of_reviews_flag" to 1 or 2, respectively.

In [35]:
def check_integer(x,start=1,end=365):
    try:
        temp_x = int(x)
        return start <= temp_x <= end
    except:
        return False

In [36]:
def clean_minimum_nights(df):
    #raise Exception("not yet have implementation")
    # do something here
    print("not yet have implementation")
        
    return df

In [37]:
airbnb_pd = clean_minimum_nights(airbnb_pd)

not yet have implementation


# Minimum nights checking

This query should return zero rows once you implement the cleaning process

In [38]:
min_check_pd = airbnb_pd[airbnb_pd.minimum_nights_flag==0]
min_check_pd = min_check_pd[min_check_pd.minimum_nights.apply(lambda x:check_integer(x,1,365))==False]
min_check_pd[["id","minimum_nights"]]

,id,minimum_nights
10,1027463,C32
19,2320027,NaN
40,5185790,3v2
103,11609854,L32
166,15758556,B60
175,16508819,NaN
237,20995084,3V2
299,24947013,3F2
336,27016482,3a2
385,29256818,3E2


In [39]:
def clean_number_of_reviews(df):
    #raise Exception("not yet have implementation")
    # do something here
    print("not yet have implementation")
        
    return df

In [40]:
airbnb_pd = clean_number_of_reviews(airbnb_pd)

not yet have implementation


# Clean number of reviews checking

This query should return zero rows once you implement the cleaning process

In [41]:
min_check_pd = airbnb_pd[airbnb_pd.number_of_reviews_flag==0]
min_check_pd = min_check_pd[min_check_pd.number_of_reviews.apply(lambda x:check_integer(x,0,999999))==False]
min_check_pd[["id","number_of_reviews"]]

,id,number_of_reviews
1,84042,L124
3,264547,6d7
14,1921670,2U87
41,5205377,2a56
81,8802052,L329
...,...,...
1250,645517682076041188,o32
1253,649974943915158739,1G2
1258,651103814026388825,NaN
1335,689796024545543021,1k7


# cleanup last_review

The "last_review" column should be in the format of ISO-date (yyyy-mm-dd). We have provided a "check_date" function to verify the date format.

If a value is outside the date format or is null and you are unsure how to handle it, you can flag the row for deletion by setting the "last_review_flag" to 1 or 2.


In [42]:
from datetime import datetime
def check_date(x,fmt="%Y-%m-%d"):
    try:
        datetime.strptime(x,fmt)
        return True
    except:
        return False

In [43]:
def clean_last_reviews(df):
    #raise Exception("not yet have implementation")
    # do something here
    print("not yet have implementation")
        
    return df

In [44]:
airbnb_pd = clean_last_reviews(airbnb_pd)

not yet have implementation


# Last Review checking

This query should return zero rows once you implement the cleaning process

In [45]:
last_review_check_pd = airbnb_pd[airbnb_pd.last_review_flag==0]
last_review_check_pd = last_review_check_pd[last_review_check_pd.last_review.apply(lambda x:check_date(x))==False]
last_review_check_pd[["id","last_review"]]

,id,last_review
40,5185790,NaN
69,7713352,10/28/22
100,10634059,NaN
107,12136030,NaN
112,12448898,NaN
...,...,...
1530,778024630376031823,NaN
1534,781914893885673817,NaN
1535,781914893885673817,NaN
1540,785423932330914663,NaN


# save the dataset to csv

In [46]:
airbnb_pd.to_csv("chicago_vert_dataset_cleaned_3.csv")

# columns that potentially will be used for analysis:
id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,price

In [47]:
columns_used = ["id","name","host_id","host_name",
                         "neighbourhood","latitude","longitude",
                         "room_type","minimum_nights","number_of_reviews","last_review","price"]

In [48]:
airbnb_pd[columns_used]

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,price
0,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,"Chicago, Illinois, United States",41.901660,-87.680210,Entire home/apt,2,483,2023-02-26,$85.00
1,84042,The Explorer Room,334241,Michael And Veronica,"Chicago, Illinois, United States",41.896150,-87.679340,Private room,3,L124,2022-12-11,$89.00
2,145659,Trendy Roscoe Village 3BR/2BR walk to shops,683529,Joe,"Chicago, Illinois, United States",41.943420,-87.681210,Entire home/apt,2,59,2022-12-31,$198.00
3,264547,Steps From the Best Chicago Offers,1387948,Dominic,"Chicago, Illinois, United States",41.884130,-87.643150,Entire home/apt,32,6d7,2021-10-17,$204.00
4,290155,15 Minutes to Everything. Skyline View. Quiet.,1505878,Bob,"Chicago, Illinois, United States",41.827120,-87.640460,Entire home/apt,32,50,2020-03-17,$130.00
...,...,...,...,...,...,...,...,...,...,...,...,...
1537,782137041139805370,UIC / Little Italy - Adorable 2 Bed 1 Bath Apt.,215444392,Jenna,NaN,41.870469,-87.662579,Entire home/apt,2,6,2023-03-05,$130.00
1538,782628636832878491,"Steps to Shop, Eat, Train | Easy Access | Zencity",47172572,Zencity,"Chicago, Illinois, United States",41.896820,-87.661130,Entire home/apt,1,8,2023-03-04,$81.00
1539,782643895516370805,Old Town Oasis,169297663,William,NaN,41.901460,-87.636660,Entire home/apt,3,7,2023-03-05,$336.00
1540,785423932330914663,"River North 1br w/ gym, pool & roof, nr Riverwalk",107434423,Blueground,"Chicago, Illinois, United States",41.890516,-87.635955,Entire home/apt,32,0,NaN,$158.00


In [49]:
# save the csv
airbnb_pd.latitude_flag

0       0
1       0
2       0
3       0
4       0
       ..
1537    0
1538    0
1539    0
1540    0
1541    0
Name: latitude_flag, Length: 1542, dtype: int64

In [50]:
airbnb_pd.groupby("latitude_flag").count()

,Unnamed: 0,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,...,calculated_host_listings_count_shared_rooms,reviews_per_month,id_flag,host_id_flag,neighbourhood_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
latitude_flag,,,,,,,,,,,,,,,,,,,,,
0,1535,1535,1535,1535,1535,1535,1533,1535,1107,1535,...,1535,1267,1535,1535,1535,1535,1535,1535,1535,1535
1,7,7,7,7,7,7,7,7,4,7,...,7,6,7,7,7,7,7,7,7,7
